In [91]:
"""this scripts fixes the encoding of some non-UTF characters that are saved with the
Numeric character reference (for example: ç was saved as &#231;)
Thankfully, the html library can unescape these characters for so we can write the
text back as UTF
"""
from html import unescape
from io import StringIO
import pandas as pd
import numpy as np
import re

inputFile = 'pik_edoc_20190404.csv'
outputFile = 'data_formatted.csv'

data = None
with open(inputFile, 'r', encoding='UTF-8') as file:
    data = file.read()
    print(inputFile + ' loaded')

# unescape to fix the encoding of some chars
data = StringIO(unescape(data))
print('unescaped')

#read string as csv file
df = pd.read_csv(data, encoding="utf8", sep=',')

#remove last column, its empty
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

#replace all \N with NaN (pandas like it this way)
df.replace("\\N", np.nan, inplace=True)

#some authors and editors names have a : at the end, remove that
def RemoveColon(string):
    return string.replace(':', '')

df['authors'] = df['authors'].astype(str).apply(RemoveColon)
df['editors'] = df['editors'].astype(str).apply(RemoveColon)


"""
as per Email from PIK:
Das was unter "keywords" steht sind die
alten Abteilungsnamen, die bis vor der Evaluation des Instituts in den 2000ern Bestand
hatten. Danach wurden neue Domainnamen gebildet - die ab diesem Zeitpunkt im Feld x9
hinterlegt sind. Tats?chliche Keywords liegen also, wenn vorhanden, in Feld "x1". 
however, the field x9 does not exist in the data we received
"""
df.rename(columns={
'keywords': 'oldDepartmentNames',
'x1 ( =Feld "Keyword";  u.a. belegt mit Info zu peer-review, wenn kein ISI-Journal)':'keywordsAndPeerReview',
'x4 ( = DOI / Identifier)':'DOI',
'relation (= Serie)': 'series'
}, inplace=True)



print("formatted")
df.to_csv(outputFile, encoding="utf8", index=False)

print(outputFile, "saved")

pik_edoc_20190404.csv loaded
unescaped
formatted
data_formatted.csv saved


In [92]:
df_copy = pd.read_csv('data_formatted.csv', sep=',')

In [93]:
# drop rows that contain status 'submitted' in column 'year'
df_copy = df_copy[~df_copy['year'].str.contains('submitted', na=False)]

In [94]:
df_copy = df_copy[~df_copy['year'].str.contains('Submitted', na=False)]

In [95]:
# drop row that contains 'Marburg' in column 'year'
df_copy=df_copy.drop([3238])

In [99]:
# extract numbers of entries in 'year' except for 'submitted' entries
df_years=df_copy
year = []
for _, row in df_years.iterrows():
    _id = row['id']
    names = str(row['year']).split(' ')
    for name in names:
        year.append([_id, name.strip()])
year = pd.DataFrame(year, columns=['id', 'year'])

In [101]:
df_years=year[~year['year'].str.contains('[A-Z]', regex=True, flags= re.I)] #.to_csv("year", sep='\t', encoding='utf-8')

In [102]:
# get list of rows with NaN in column'year'
df_nan=pd.read_csv('data_formatted.csv', sep=',')
df_nan=df_temp2.loc[df_nan['year'].isnull()]

In [103]:
df_nan= pd.DataFrame(df_nan, columns=['id', 'year'])

In [105]:
# concatenate table of NaN entries in column 'year' and table with numerical entry
df_concat=pd.concat([df_years, df_nan], ignore_index=True, sort=True)

In [106]:
# join tables on id, add cleaned 'year' data to complete table
result = pd.merge(df_copy,
                 df_concat[['id','year']],
                 on='id')

In [107]:
# drop obsolete 'year' column
result=result.drop(columns=['year_x'])

In [108]:
#rename column
result=result.rename(index=str, columns={"year_y": "year"})

In [110]:
# replace values of '08/11/07' structure
result.at['3506', 'year'] = 2007
result.at['3508', 'year'] = 2007
result.at['3509', 'year'] = 2007
result.at['3511', 'year'] = 2007
result.at['3512', 'year'] = 2007
result.at['3514', 'year'] = 2007
result.at['3583', 'year'] = 2007
result.at['3600', 'year'] = 2007
result.at['3604', 'year'] = 2008
result.at['7861', 'year'] = 2017
result.at['7862', 'year'] = 2017

# replace wrong values 
result.at['7923', 'year'] = 2018
result.at['5588', 'year'] = 2012
result.at['8244', 'year'] = 2018

# replace wrong value of '16'
result.at['6874', 'year']= 2015

# drop rows having '9.' as entry in column 'year'
result=result.drop(index='7926')
result=result.drop(index='7924')
result=result.drop(index='7922')
result=result.drop(index='7401')

In [90]:
# check whether 'id' is still unique
#result.groupby('id').count().sort_values('id', ascending=False)

In [112]:
outputFile = 'year_cleaned.csv'
result.to_csv(outputFile, encoding="utf8", index=False)
print(outputFile, "saved")

year_cleaned.csv saved
